In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import cv2
import glob
import sys
import os
import shutil 
import math
import numpy as np
from tqdm import tqdm

from skimage.transform import resize
from sklearn.model_selection import train_test_split
from scipy import stats as s

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import Dense, InputLayer, Dropout, Flatten, Activation, GRU, Conv1D, Conv3D, MaxPooling3D
from keras.layers import Conv2D, MaxPooling2D, MaxPooling1D, GlobalMaxPooling2D, Convolution2D, ZeroPadding2D, Bidirectional, TimeDistributed
from keras.layers.recurrent import LSTM
from keras.utils import np_utils
from keras.preprocessing import image, sequence
from keras.callbacks import ModelCheckpoint


#from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import optimizers

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [3]:
train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
print(classes)
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
print(word_ids)

['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'M01', 'M04']
['F10', 'M07']
['F11', 'M08']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']


In [4]:
files = glob.glob('train_MIRACL/*')
for file_rem in files:
    os.remove(file_rem)

counter = 0
for classi in classes:
    for person in tqdm(train_people):
        for word_id in word_ids:
            for f in sorted(glob.glob(os.path.join('MIRACL/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (128-(abs(x1-x2)))/2
                    offset_y = (128-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                cv2.imwrite('train_MIRACL/' + classi + '_' 
                            + person + '_' 
                            + word_id + '_' 
                            + 'frame' + f[29:-4] + '.jpg', img)

100%|██████████| 10/10 [00:57<00:00,  5.79s/it]


In [5]:
files = glob.glob('val_MIRACL/*')
for file_rem in files:
    os.remove(file_rem)

counter = 0
for classi in classes:
    for person in tqdm(val_people):
        for word_id in word_ids:
            for f in sorted(glob.glob(os.path.join('MIRACL/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (128-(abs(x1-x2)))/2
                    offset_y = (128-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                cv2.imwrite('val_MIRACL/' + classi + '_' 
                            + person + '_' 
                            + word_id + '_' 
                            + 'frame' + f[29:-4] + '.jpg', img)

100%|██████████| 2/2 [00:07<00:00,  3.78s/it]


In [4]:
images = sorted(glob.glob("train_MIRACL/*.jpg"))
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    train_image.append(images[i].split('/')[1])
    train_class.append(images[i].split('/')[1][:2])
    
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('train_new.csv',header=True, index=False)

100%|██████████| 10952/10952 [00:00<00:00, 713746.60it/s]


In [5]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,01_F01_01_frame001.jpg,1
1,01_F01_01_frame002.jpg,1
2,01_F01_01_frame003.jpg,1
3,01_F01_01_frame004.jpg,1
4,01_F01_01_frame005.jpg,1


In [6]:
train.tail()

,image,class
10947,10_M04_10_frame007.jpg,10
10948,10_M04_10_frame008.jpg,10
10949,10_M04_10_frame009.jpg,10
10950,10_M04_10_frame010.jpg,10
10951,10_M04_10_frame011.jpg,10


In [7]:
images = sorted(glob.glob("val_MIRACL/*.jpg"))
val_image = []
val_class = []
for i in tqdm(range(len(images))):
    val_image.append(images[i].split('/')[1])
    val_class.append(images[i].split('/')[1][:2])
    
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('val_new.csv',header=True, index=False)

100%|██████████| 1746/1746 [00:00<00:00, 412415.09it/s]


In [8]:
val = pd.read_csv('val_new.csv')
val.head()

,image,class
0,01_F10_01_frame001.jpg,1
1,01_F10_01_frame002.jpg,1
2,01_F10_01_frame003.jpg,1
3,01_F10_01_frame004.jpg,1
4,01_F10_01_frame005.jpg,1


In [9]:
val.tail()

,image,class
1741,10_M07_10_frame004.jpg,10
1742,10_M07_10_frame005.jpg,10
1743,10_M07_10_frame006.jpg,10
1744,10_M07_10_frame007.jpg,10
1745,10_M07_10_frame008.jpg,10


In [10]:
X_tr = []

for i in tqdm(range(train.shape[0])):
    img = image.load_img('train_MIRACL/'+train['image'][i], target_size=(128,128,3))
    img = image.img_to_array(img)
    img = keras.applications.mobilenet.preprocess_input(img)
    
    X_tr.append(img)

100%|██████████| 10952/10952 [00:13<00:00, 824.26it/s]


In [11]:
X_tr = np.array(X_tr)
print(X_tr.shape)

(10952, 128, 128, 3)


In [12]:
X_vl = []

for i in tqdm(range(val.shape[0])):
    img = image.load_img('val_MIRACL/'+val['image'][i], target_size=(128,128,3))
    img = image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    
    X_vl.append(img)

100%|██████████| 1746/1746 [00:02<00:00, 862.93it/s]


In [13]:
X_vl = np.array(X_vl)
print(X_vl.shape)

(1746, 128, 128, 3)


In [14]:
import sys
print(sys.getsizeof(X_tr)*1e-6)
print(sys.getsizeof(X_vl)*1e-6)

2153.25096
343.277712


In [15]:
y_tr = train['class']
y_vl = val['class']

In [16]:
#X_tr_rnn = np.reshape(X_tr, (X_tr.shape[0], 1, X_tr.shape[1]))
#X_vl_rnn = np.reshape(X_vl, (X_vl.shape[0], 1, X_vl.shape[1]))
y_tr_rnn = pd.get_dummies(y_tr)
y_vl_rnn = pd.get_dummies(y_vl)
#print(X_tr_rnn.shape)
#print(X_vl_rnn.shape)
print(y_tr_rnn.shape)
print(y_vl_rnn.shape)

(10952, 10)
(1746, 10)


In [20]:
model = Sequential()

model.add(
    TimeDistributed(
        Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu'),
        input_shape=(None, 128, 128, 3)
    ))

model.add(
    TimeDistributed(
        MaxPooling2D(pool_size=(2,2))
    ))

model.add(
    TimeDistributed(
        Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu')
    ))

model.add(
    TimeDistributed(
        MaxPooling2D(pool_size=(2,2))
    ))

model.add(
    TimeDistributed(
        Flatten()
    ))

model.add(LSTM(100))

model.add(Dense(10, activation='softmax'))

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_9 (TimeDist (None, None, 126, 126, 32 896       
_________________________________________________________________
time_distributed_10 (TimeDis (None, None, 63, 63, 32)  0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, None, 61, 61, 32)  9248      
_________________________________________________________________
time_distributed_12 (TimeDis (None, None, 30, 30, 32)  0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, None, 28800)       0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               11560400  
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total para

In [22]:
save_weights = ModelCheckpoint('Custom_RNN_top_weights.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [23]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='Adam', metrics=['accuracy'])

In [24]:
# got to train in batches every time step thus pass an input with shape (batch_size, frames, height, width, channels)
# here i got the shape of (frames, height, width, channels)
# TODO: generate batches somehow

model.fit(X_tr, y_tr_rnn, epochs=1, validation_data=(X_vl, y_vl_rnn), callbacks=[save_weights], batch_size=128)

ValueError: Error when checking input: expected time_distributed_9_input to have 5 dimensions, but got array with shape (10952, 128, 128, 3)

In [ ]:
files = glob.glob('test_MIRACL/*')
for file_rem in files:
    os.remove(file_rem)

counter = 0
for classi in classes:
    for person in tqdm(test_people):
        for word_id in word_ids:
            for f in sorted(glob.glob(os.path.join('MIRACL/'+person+'/words/'+classi+'/'+ word_id, "*.jpg"))):
                img = cv2.imread(f, 1)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                
                for k, rect in enumerate(rects):
                    x1 = rect.left()
                    y1 = rect.top()
                    x2 = rect.right()
                    y2 = rect.bottom()
                    
                    offset_x = (128-(abs(x1-x2)))/2
                    offset_y = (128-(abs(y1-y2)))/2
                    
                    img = img[int(y1-offset_y):int(y2+offset_y), int(x1-offset_x):int(x2+offset_x)]
                
                counter += 1
                
                cv2.imwrite('test_MIRACL/' + classi + '_' 
                            + person + '_' 
                            + word_id + '_' 
                            + 'frame' + f[29:-4] + '.jpg', img)

In [ ]:
images = glob.glob("test_MIRACL/*.jpg")
test_image = []
test_class = []
for i in tqdm(range(len(images))):
    test_image.append(sorted(images)[i].split('/')[1])
    test_class.append(sorted(images)[i].split('/')[1][:2])


test_data = pd.DataFrame()
test_data['image'] = test_image
test_data['class'] = test_class
test_images = test_data['image']

test_data.to_csv('test_new.csv',header=True, index=False)

In [ ]:
test = pd.read_csv('test_new.csv')
y_test = test['class']
y_test = pd.get_dummies(y_test)

test.head()

In [ ]:
video_names = sorted(list(set([test_images[i].split('.')[0][0:9] for i in range(len(test_images))])))
test_videos = pd.DataFrame(video_names)

test_videos.columns = ['dummy']
test_videos = test_videos['dummy']

print(test_videos.shape[0])
test_videos.head()

In [ ]:
print(test_images.shape[0])
test_images.tail()

In [ ]:
predict = []
actual = []

for video_id in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[video_id]
    
    files = glob.glob('temp/*')
    for f in files:
        os.remove(f)
    
    for frame_id in range(test_images.shape[0]):
        frame = test_images[frame_id]
        if test_images[frame_id].split('.')[0][0:9] == videoFile:
            img = shutil.copyfile('test_MIRACL/'+test_images[frame_id], 'temp/'+test_images[frame_id]) 
    
    images = glob.glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(128,128,3))
        img = image.img_to_array(img)
        img = keras.applications.mobilenet.preprocess_input(img)
        
        prediction_images.append(img)
        
    prediction_images = np.array(prediction_images)
    
    # TODO: fix the code below to apply for the new model
    
    prediction_images = base_model.predict(prediction_images)
    
    prediction_images = np.reshape(prediction_images, 
                                   (prediction_images.shape[0], 1, prediction_images.shape[1])) #(x, 1, 1024)
    
    prediction = model.predict_classes(prediction_images)
    predict.append(y_test.columns.values[s.mode(prediction)[0][0]])
    
    actual.append(int(videoFile.split('_')[0].lstrip('0')))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100